# Use case for router chain


- A student asking physics question
    - "How does a magnet work?" - Beginner question
    - "Explain Feynman diagram?" - Expert question
    
Setup
- INPUT -> ROUTER -> LLM Deides chain -> Chain -> OUTPUT

### Prompt Setup

In [3]:
beginner_template = '''You are a physics teacher, who is really focused on beginners, and explain complex concept in simple to understand terms. You assume no prior knowledge. 
Here is your question:\n{input}
'''

In [4]:
expert_template = '''You are a physics professor, who explains physics topics to advanced audience members. You can assume anyone your answer has a PhD in physics. 
Here is your question:\n{input}
'''

### ROUTE PROMPT INFORMATION

[] This takes NAME, DESCRIPTION, and TEMPLATE

In [6]:
prompt_infos = [
    {
        'name': 'beginner physics',
        'description': 'Answers basic physics questions',
        'template': beginner_template
    },
    {
        'name': 'advanced physics',
        'description': 'Answers advanced physics questions',
        'template': expert_template
    }
]

### Chain setup

In [8]:
import os

api_key = open('./openai_key.txt').read()
os.environ['OPENAI_API_KEY'] = api_key

In [9]:
from langchain_openai import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.chains import LLMChain


In [10]:
chat = ChatOpenAI()

In [11]:
destination_chains = {}

for p_info in prompt_infos:
    name = p_info['name']
    prompt_template = p_info['template']
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=chat, prompt=prompt)
    destination_chains[name] = chain
    

In [12]:
destination_chains.keys()

dict_keys(['beginner physics', 'advanced physics'])

In [13]:
default_prompt = ChatPromptTemplate.from_template('{input}')
default_chain = LLMChain(llm=chat, prompt=default_prompt)

### Setup multi prompt routing

In [15]:
from langchain.chains.router.multi_prompt_prompt import  MULTI_PROMPT_ROUTER_TEMPLATE

In [16]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [17]:
destinations = [ f"{p['name']}: {p['description']}"  for p in prompt_infos ]

In [18]:
destinations

['beginner physics: Answers basic physics questions',
 'advanced physics: Answers advanced physics questions']

In [19]:
# Join this a giant string
destination_str = '\n'.join(destinations)

In [20]:
print(destination_str)

beginner physics: Answers basic physics questions
advanced physics: Answers advanced physics questions


### Router Prompts

In [22]:
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser

In [23]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destination_str)

In [24]:
# Note how << CANDIDATE PROMPTS >> now has out destinations
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
beginner physics: Answers basic physics questions
advanced physics: Answers advanced physics question

In [25]:
router_prompt = PromptTemplate(template=router_template,
                               input_variables=['input'],
                               output_parser=RouterOutputParser())

### Final Chain

In [27]:
from langchain.chains.router import MultiPromptChain

In [28]:
router_chain = LLMRouterChain.from_llm(chat, router_prompt)

In [29]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain,
                         verbose=True)

In [30]:
result1 = chain.run("How do magenets work")

C:\ProgramData\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
C:\ProgramData\anaconda3\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(




> Entering new MultiPromptChain chain...
beginner physics: {'input': 'How do magnets work'}
> Finished chain.


In [31]:
print(result1)

Magnets are really fascinating objects that have the power to attract or repel certain materials. They work because of something called magnetism. 

Inside a magnet, there are tiny particles called electrons that are constantly moving. These electrons have a property called "spin" which creates a magnetic field around them. Think of this magnetic field as an invisible force that extends out from the magnet.

When you bring another object close to a magnet, like a piece of iron, the magnetic field interacts with the electrons in that object. The magnet's magnetic field aligns the electrons in the iron in a specific way.

If the electrons in the iron align in the same direction as the electrons in the magnet, they will attract each other. It's like they become friends and stick together. This is why the iron gets attracted to the magnet and sticks to it.

On the other hand, if the electrons in the iron align in the opposite direction to the electrons in the magnet, they will repel each o

In [32]:
result2 = chain.run("Please explain Quantum Gravity")



> Entering new MultiPromptChain chain...


C:\ProgramData\anaconda3\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


advanced physics: {'input': 'Please explain Quantum Gravity'}
> Finished chain.


In [33]:
print(result2)

Quantum gravity is a theoretical framework that aims to reconcile two fundamental theories in physics: quantum mechanics, which describes the behavior of particles on a very small scale, and general relativity, which explains the behavior of gravity on a large scale.

In classical physics, including general relativity, gravity is described as a geometric property of spacetime. It is the curvature of spacetime caused by mass and energy that influences the motion of objects. On the other hand, quantum mechanics describes the behavior of particles and forces at the microscopic level, where the discrete nature of energy and the uncertainty principle play a significant role.

Quantum gravity attempts to extend these principles to a unified theory that can explain the behavior of gravity at both the macroscopic and microscopic levels. However, it is important to note that a complete and fully satisfactory theory of quantum gravity has not yet been established. Nonetheless, several theoretica